In [15]:


import numpy as np
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
from nilearn.image import index_img
from sklearn.preprocessing import StandardScaler
import nibabel as nib
# from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit
# from skmultilearn.model_selection import iterative_train_test_split
# from verstack.stratified_continuous_split import scsplit
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
import pandas as pd
import matplotlib.pyplot as plt

brain_data = datasets.fetch_oasis_vbm(n_subjects=100)
crad = datasets.fetch_atlas_craddock_2012()

In [16]:
def plot_each_n_rois(i):
  atlas_nii = index_img(crad['scorr_mean'], i) # this needs to be modified for some tasks
  print('%i regions found in this atlas' % (len(np.unique(atlas_nii.get_data().ravel())) - 1))
  masker = NiftiLabelsMasker(labels_img= atlas_nii, standardize=True)
  input_variables = masker.fit_transform(brain_data.gray_matter_maps)
  output_variable = StandardScaler().fit_transform(brain_data.ext_vars.age[:, None])[:, 0] # gives subject age on standard units after z-scoring

  inpu=pd.DataFrame(input_variables)
  outpu=pd.DataFrame(output_variable)
  ############## note no bootstrap, just taking the first 80% of 100 subjects, that is the first 80 subjects as training set ############

  # train_input = input_variables[:80]
  # train_output = output_variable[:80]

  # test_input = input_variables[80:]
  # test_output = output_variable[80:]

  kf = StratifiedShuffleSplit(n_splits=50, train_size = 0.8)

  accuracy = 0
  j = 0
  
  for train_index,test_index in kf.split(input_variables,output_variable):
    X_train, X_test = input_variables[train_index], input_variables[test_index]
    y_train, y_test = output_variable[train_index], output_variable[test_index]

    # X_train, X_test, y_train, y_test = scsplit(inpu, outpu, stratify=outpu, train_size=0.8, test_size=0.2, random_state=i)
    KNN = KNeighborsRegressor(n_neighbors=5).fit(X_train, y_train)
    KNN_predict_output = KNN.predict(X_test)
    j += 1
    KNN_accuracy = r2_score(y_test, KNN_predict_output)
    accuracy += KNN_accuracy

  return i, accuracy/j

x_list = [] ### n_rois
# log_y_list = [] ### accuracy
KNN_y_list = []

for i in range (43):
  i, KNN_accuracy = plot_each_n_rois(i)

  # print("i: ", i,"accuracy: ", accuracy)
  x_list.append(i)
  # log_y_list.append(log_accuracy)
  KNN_y_list.append(KNN_accuracy)

print(x_list)
# print(log_y_list)
print(KNN_y_list)
plt.xticks(x_list)
# plt.plot(x_list,log_y_list,label='LogReg')
plt.plot(x_list,KNN_y_list,label='KNN')

# plt.legend()
plt.show()

10 regions found in this atlas


ValueError: ignored